## 6.01 - Supervised Learning Model Comparison

Recall the "data science process."

1. Define the problem.
2. Gather the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. When predicting `e401k`, you may use the entire dataframe if you wish.

### Step 2: Gather the data.

##### 1. Read in the data from the repository.

In [28]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_csv('401ksubs.csv')

In [3]:
df['inc_highlow'] = np.where(df['inc'] < 50, 0, 1)

In [4]:
df.head(10)

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq,inc_highlow
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600,0
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225,1
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936,0
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936,1
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809,0
5,0,15.000,1,0,60,3,0.000,0,0,225.0000,3600,0
6,0,37.155,1,0,49,5,3.483,0,1,1380.4940,2401,0
7,0,31.896,1,0,38,5,-2.100,0,0,1017.3550,1444,0
8,0,47.295,1,0,52,2,5.290,0,1,2236.8170,2704,0
9,1,29.100,0,1,45,1,29.600,0,1,846.8100,2025,0


In [6]:
data = df[['marr', 'male', 'age', 'fsize', 'nettfa', 'agesq']]
lin_target = df['inc']
cat_target = df['inc_highlow']

##### 2. What are 2-3 other variables that, if available, would be helpful to have?

Total household income
Credit score

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

Because you shouldn't be judging people based on race - something they have no control over.  People should be ethically judged based on factors they control.

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

Married should have no effect on income

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs might have done this!

Age squared and inc squared
Age has an incredibly high correlation with income, and the value in a 401k.  It would make sense to try and put more emphasis on this variable.
Income is probably the msot important varaible for determining 401k eligability.  Same goes to put more emphasis on the variable.

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

There seem to be two descriptions that are off, both age and inc on their regular columns show as age^2 and inc^2 respecitively, when there are already those two columns.

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

linear regression - makes sense based on the major factors of age and assest value

logisitc regression - can also have use but perhaps less than a linear regression

k-nearest neighbors - ???

decision tree - probably the most effective given the large number of binary information points

random forest - same as above


##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector regressor
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [8]:
X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(data, lin_target, random_state=42, test_size=0.3)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data, cat_target, random_state=42, test_size=0.3)

In [9]:
X_train_lin.head()

,marr,male,age,fsize,nettfa,agesq
1097,1,0,37,5,-11.635,1369
548,1,0,32,4,3.500,1024
5885,0,0,31,2,6.442,961
3290,1,0,49,4,388.000,2401
3653,0,0,61,5,1.000,3721


In [10]:
X_train_cat.head()

,marr,male,age,fsize,nettfa,agesq
1097,1,0,37,5,-11.635,1369
548,1,0,32,4,3.500,1024
5885,0,0,31,2,6.442,961
3290,1,0,49,4,388.000,2401
3653,0,0,61,5,1.000,3721


In [11]:
lin_reg = LinearRegression()
log_reg = LogisticRegression()
rfc = RandomForestClassifier()
bc = BaggingClassifier()
svc = SVC()
knn = KNeighborsClassifier()
ss = StandardScaler()

In [25]:
ss.fit(X_train_lin)
X_train_lin_sc = ss.transform(X_train_lin)
X_test_lin_sc = ss.transform(X_test_lin)

In [26]:
ss.fit(X_train_cat)
X_train_cat_sc = ss.transform(X_train_cat)
X_test_cat_sc = ss.transform(X_test_cat)

In [32]:
lin_reg.fit(X_train_lin, y_train_lin)
print("Linear Regression Train Score:", lin_reg.score(X_train_lin, y_train_lin).round(3))
print("Linear Regression Test Score:", lin_reg.score(X_test_lin, y_test_lin).round(3))

Linear Regression Train Score: 0.301
Linear Regression Test Score: 0.261


In [14]:
lin_reg.fit(X_train_lin_sc, y_train_lin)
print(lin_reg.score(X_train_lin_sc, y_train_lin))
print(lin_reg.score(X_test_lin_sc, y_test_lin))

0.30065902255610066
0.26103635936705893


In [34]:
knn_score = cross_val_score(knn, X_train_cat_sc, y_train_cat, cv=5)
print("{} Score:\t{:0.3} ± {:0.3}".format("K-Nearest Neighbors", knn_score.mean().round(3), knn_score.std().round(3)))

K-Nearest Neighbors Score:	0.781 ± 0.015


In [35]:
rfc_score = cross_val_score(rfc, X_train_cat_sc, y_train_cat, cv=5)
print("{} Score:\t{:0.3} ± {:0.3}".format("Random Forest", rfc_score.mean().round(3), rfc_score.std().round(3)))

Random Forest Score:	0.763 ± 0.006


In [36]:
bc_score = cross_val_score(bc, X_train_cat_sc, y_train_cat, cv=5)
print("{} Score:\t{:0.3} ± {:0.3}".format("Bagging Classifier", bc_score.mean().round(3), bc_score.std().round(3)))

Bagging Classifier Score:	0.76 ± 0.009


In [37]:
svc_score = cross_val_score(svc, X_train_cat_sc, y_train_cat, cv=5)
print("{} Score:\t{:0.3} ± {:0.3}".format("Support Vector Classifier", svc_score.mean().round(3), svc_score.std().round(3)))

Support Vector Classifier Score:	0.795 ± 0.008


##### 9. What is bootstrapping?

A testing method that relies on sampling with replacements - used to make up for weaknesses in data such as unbalanced classes.

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

The bagged decision trees are bootstrapped - that is shown different subsets of the data.  A classic decision tree is shown the entirity of the data (test data that is).

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

A random forest is a set of bagged decision trees that have been "decorrelated" from each other via random selection of the sub features.

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

For the exact reason that you would use a random forest - to remove correlation from the data and strenghten the model.  It lowers variance.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a $k$-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector classifier
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.